# Welcome to PYNQ

## Getting Started

To get started using PYNQ, try running the example notebooks in the folders described below. 

* **getting_started**: includes an introduction to using Jupyter notebook with PYNQ, the Python environment, and how to use some basic features of the curernt platform. 

* **common**: contains example notebooks on how to download an overlay, how to set the Zynq clocks, how to execute Linux shell commands, and how to use USB devices.

If other overlays or packages are installed, other folders with example notebooks may also be available in this directory.  


## Documentation

Please see the latest <a href="http://pynq.readthedocs.io">PYNQ Documentation on readthedocs</a>.  


## Support

For questions or support, go to the forum on the <a href="http://www.pynq.io">PYNQ project webpage </a>.


## Project webpage

You can find details on the <a href="http://www.pynq.io">PYNQ project webpage </a>.


## GitHub

The PYNQ Repository is hosted on github: <a href="https://github.com/Xilinx/PYNQ">PYNQ GitHub Repository</a>.

In [1]:
# import pyspark
# import pynq
import findspark
findspark.init()


In [2]:

import pyspark

In [3]:
from pyspark.sql import SparkSession

In [4]:
import os

In [5]:
os.getcwd()

'/home/xilinx/jupyter_notebooks'

In [45]:
from pynq import Overlay, Clocks

print(f'CPU:   {Clocks.cpu_mhz:.6f}MHz')
print(f'FCLK0: {Clocks.fclk0_mhz:.6f}MHz')


CPU:   650.000000MHz
FCLK0: 76.923077MHz


In [51]:
# Use bitstream for PL 
overlay = Overlay(os.getcwd() + "/nn3_stream/dpu_ip/dpu.bit")

In [10]:
overlay?

In [52]:
# DPU IP
dpu_ip = overlay.dpu_ip
dpu_ip?

In [53]:
# DMA IP
dma = overlay.axi_dma
dma?

In [54]:
#  Shows the Registers we need to access - Can use access via names or direct memory  (via names is easier)
dpu_ip.register_map


RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED=0),
  size = Register(size=0, RESERVED=0),
  size_ctrl = Register(size_ap_vld=0, RESERVED=0)
}

In [55]:
# NN3 Stream

# First signal to set high. Ensures AP_START Signal does not go low after one cycle
# In AXI_STREAM, only setting AP_START enables computations for 1 stream
dpu_ip.register_map.CTRL.AUTO_RESTART = 1
# Computations occur while high
dpu_ip.register_map.CTRL.AP_START = 1

In [56]:
# Check if signals were set and read only registers have values
dpu_ip.register_map

RegisterMap {
  CTRL = Register(AP_START=1, AP_DONE=0, AP_IDLE=0, AP_READY=0, RESERVED_1=0, AUTO_RESTART=1, RESERVED_2=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED=0),
  size = Register(size=10, RESERVED=0),
  size_ctrl = Register(size_ap_vld=1, RESERVED=0)
}

In [57]:
size = int(dpu_ip.register_map.size)
print(size)

10


In [33]:
################# Test 1: Feed a single stream ##################

import numpy as np
from pynq import allocate

row_length = 2
inStream = allocate(shape=(size,), dtype=np.uint32)
outStream = allocate(shape=(size//row_length,), dtype=np.uint32)

for i in range(0, size, row_length):
    inStream[i] = 1587568890
    inStream[i+1] = 1
    
inStream.flush()
print("InStream: ", inStream)
print("OutStream: ", outStream)

InStream:  [1587568890          1 1587568890          1 1587568890          1
 1587568890          1 1587568890          1]
OutStream:  [0 0 0 0 0]


In [34]:
dma.sendchannel.transfer(inStream)
print("Completed transfer inStream")

Completed transfer inStream


In [35]:
dma.recvchannel.transfer(outStream)
print("Completed transfer outStream")

Completed transfer outStream


In [36]:
import time
# Call if TLAST is not set high in HLS code
startTime = time.time()
dma.sendchannel.wait()
# Call if TLAST is not set high in HLS code
dma.recvchannel.wait()
endTime = time.time()

In [37]:
print("Processor waits on dma for: ", endTime-startTime)

Processor waits on dma for:  0.0010082721710205078


In [38]:
outStream.flush()
print(outStream)

[105 105 105 105 105]


In [39]:
inStream.freebuffer()
print("Freed inStream buffer")


Freed inStream buffer


In [40]:
outStream.freebuffer()
print("Freed outStream buffer")

Freed outStream buffer


In [74]:
################# Test 2: Feed continuous streams ##################

import numpy as np
from pynq import allocate

row_length = 2
inStream = allocate(shape=(size,), dtype=np.uint32)
outStream = allocate(shape=(size//row_length,), dtype=np.uint32)

# Test Continuous Streams
for i in range(0, size, row_length):
    inStream[i] = 1587568890
    inStream[i+1] = 1
    
inStream.flush()
print("InStream: ", inStream)
print("OutStream: ", outStream)


InStream:  [1587568890          1 1587568890          1 1587568890          1
 1587568890          1 1587568890          1]
OutStream:  [0 0 0 0 0]


In [99]:
import time

streams = 100
listOut = []
startTime = time.time()
for i in range(streams):
    
    dma.sendchannel.transfer(inStream)
    
    dma.recvchannel.transfer(outStream)
    
#     if i == (streams - 1):
#         dmaStartTime = time.time()
        
    dma.sendchannel.wait()

    dma.recvchannel.wait()
    
#     if i == (streams - 1):
#         dmaEndTime = time.time()
#         print("DMA Block Time: {0}".format(dmaEndTime - dmaStartTime))
        
#     Not sure if flush is needed after IP writes to outStream
#     outStream.flush()
    
#     listOut.append(outStream)
endTime = time.time()
inStream.freebuffer()
outStream.freebuffer()
print("Completed transfer")    

Completed transfer


In [100]:
print("Time for {0} streams of {1} records in ms: {2}".format(streams, size,  endTime - startTime))

Time for 100 streams of 10 records in ms: 0.03754591941833496


In [96]:
# Check records
# for i in range(streams):
#     print(listOut[i])